# Lab 1: Create a Direct Lake Semantic Model

## Overview

This lab walks through the complete process of building a Direct Lake semantic model from scratch using Microsoft Fabric. Starting from an empty lakehouse, you will load sample data, create a semantic model, define relationships and measures, and validate the model using DMV queries.

### Workshop Flow

1. Create a lakehouse and load Adventure Works sample data
2. Create a Direct Lake semantic model from the lakehouse tables
3. Define star schema relationships between fact and dimension tables
4. Add business measures and configure the model for reporting
5. Validate model behaviour using DAX queries and DMV analysis

### Key Concepts

- **Direct Lake** allows Power BI to query data directly from Delta Lake files without importing data into the model
- **Adventure Works** is a sample business dataset containing customers, products, dates, and internet sales
- **Semantic model** is the business logic layer that sits on top of the data, providing relationships, measures, and formatting

### Learning Objectives

- Set up a Fabric lakehouse and load sample data into Delta tables
- Create a Direct Lake semantic model programmatically using Semantic Link Labs
- Define table relationships and business measures
- Use DMV queries to monitor column loading and memory usage

**Estimated duration:** 30-45 minutes

---

*Deutsche Version:*

# Lab 1: Ein Direct Lake Semantic Model erstellen

## Uebersicht

In diesem Lab wird der vollstaendige Prozess zum Erstellen eines Direct Lake Semantic Models von Grund auf mit Microsoft Fabric durchlaufen. Ausgehend von einem leeren Lakehouse laden Sie Beispieldaten, erstellen ein Semantic Model, definieren Beziehungen und Measures und validieren das Modell mithilfe von DMV-Abfragen.

### Ablauf des Workshops

1. Ein Lakehouse erstellen und Adventure Works-Beispieldaten laden
2. Ein Direct Lake Semantic Model aus den Lakehouse-Tabellen erstellen
3. Sternschema-Beziehungen zwischen Fakten- und Dimensionstabellen definieren
4. Geschaeftskennzahlen hinzufuegen und das Modell fuer das Reporting konfigurieren
5. Modellverhalten mithilfe von DAX-Abfragen und DMV-Analyse validieren

### Wichtige Konzepte

- **Direct Lake** ermoeglicht es Power BI, Daten direkt aus Delta Lake-Dateien abzufragen, ohne Daten in das Modell zu importieren
- **Adventure Works** ist ein Beispiel-Geschaeftsdatensatz mit Kunden, Produkten, Daten und Internetverkaeufen
- **Semantic Model** ist die Geschaeftslogikschicht ueber den Daten, die Beziehungen, Measures und Formatierungen bereitstellt

### Lernziele

- Ein Fabric Lakehouse einrichten und Beispieldaten in Delta-Tabellen laden
- Ein Direct Lake Semantic Model programmatisch mit Semantic Link Labs erstellen
- Tabellenbeziehungen und Geschaeftskennzahlen definieren
- DMV-Abfragen verwenden, um das Laden von Spalten und die Speichernutzung zu ueberwachen

**Geschaetzte Dauer:** 30-45 Minuten

## Step 1: Install Required Libraries

Install the Semantic Link Labs library, which provides functions for creating and managing Direct Lake semantic models in Microsoft Fabric.

---

*Installieren Sie die Semantic Link Labs-Bibliothek, die Funktionen zum Erstellen und Verwalten von Direct Lake Semantic Models in Microsoft Fabric bereitstellt.*

In [ ]:
%pip install -q semantic-link-labs

## Step 2: Import Libraries and Set Variables

Import the required Python libraries and define variables for the lakehouse name and semantic model name used throughout this lab.

---

*Importieren Sie die erforderlichen Python-Bibliotheken und definieren Sie Variablen fuer den Lakehouse-Namen und den Semantic Model-Namen, die in diesem Lab verwendet werden.*

In [ ]:
import sempy_labs as labs
from sempy import fabric
import sempy
import pandas
import json
import time

LakehouseName = "AdventureWorks"
SemanticModelName = f"{LakehouseName}_model"

## Step 3: Create or Connect to Lakehouse

Check whether the AdventureWorks lakehouse already exists. If not, create it. Then retrieve the workspace and lakehouse identifiers needed for subsequent steps.

---

*Pruefen Sie, ob das AdventureWorks Lakehouse bereits existiert. Falls nicht, erstellen Sie es. Rufen Sie dann die Workspace- und Lakehouse-Kennungen ab, die fuer die folgenden Schritte benoetigt werden.*

In [ ]:
lakehouses=labs.list_lakehouses()["Lakehouse Name"]
if LakehouseName in lakehouses.values:
    lakehouseId = notebookutils.lakehouse.getWithProperties(LakehouseName)["id"]
else:
    lakehouseId = fabric.create_lakehouse(LakehouseName)

workspaceId = notebookutils.lakehouse.getWithProperties(LakehouseName)["workspaceId"]
workspaceName = sempy.fabric.resolve_workspace_name(workspaceId)
print(f"WorkspaceId = {workspaceId}, LakehouseID = {lakehouseId}, Workspace Name = {workspaceName}")

## Step 4: Load Adventure Works Sample Data

Load four Adventure Works tables into the lakehouse: DimCustomer, DimDate, DimProduct, and FactInternetSales. The data is sourced from region-aware endpoints and written in Delta format using overwrite mode to ensure a clean starting state.

**Tables loaded:**
| Table | Approximate Rows |
|:------|:-----------------|
| DimCustomer | 18,000 |
| DimDate | 2,500 |
| DimProduct | 600 |
| FactInternetSales | 60,000 |

**Expected output:** Four "Loaded" messages followed by "Done".

---

*Laden Sie vier Adventure Works-Tabellen in das Lakehouse: DimCustomer, DimDate, DimProduct und FactInternetSales. Die Daten stammen von regionsbewussten Endpunkten und werden im Delta-Format im Ueberschreibmodus geschrieben, um einen sauberen Ausgangszustand sicherzustellen.*

*Erwartete Ausgabe: Vier "Loaded"-Meldungen gefolgt von "Done".*

In [ ]:
capacity_name = labs.get_capacity_name()

def loadDataToLakehouse(fromTable: str, toTable: str):
    """
    Optimized data loading function with improved error handling and performance.
    
    Args:
        fromTable: Source table name to read from
        toTable: Target table name to write to
    """
    try:
        # Get lakehouse properties once and reuse
        lakehouse_props = notebookutils.lakehouse.getWithProperties(LakehouseName)
        workspaceId = lakehouse_props["workspaceId"]
        lakehouseId = lakehouse_props["id"]

        # Region-aware connection string selection
        if capacity_name == "FabConUS8-P1":  # West US 3
            conn_str = "abfss://b1d61bbe-de20-4d3a-8075-b8e2eaacb868@onelake.dfs.fabric.microsoft.com/631e45c0-1243-4f42-920a-56bfe6ecdd6d/Tables"
        else:  # North Central US (default)
            conn_str = "abfss://16cf855f-3bf4-4312-a7a1-ccf5cb6a0121@onelake.dfs.fabric.microsoft.com/99ed86df-13d1-4008-a7f6-5768e53f4f85/Tables"

        # Read source data with format specification for better performance
        customer_df = spark.read.format("delta").load(f"{conn_str}/{fromTable}")
        
        # Cache the DataFrame if it will be used multiple times or is computation-heavy
        customer_df.cache()
        
        # Write with optimized settings
        (customer_df
         .write
         .format("delta")
         .mode("overwrite")
         .option("overwriteSchema", "true")
         .save(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/{toTable}"))
        
        # Unpersist cached DataFrame to free memory
        customer_df.unpersist()
        
        print(f"Loaded {toTable}")
        
    except Exception as e:
        print(f"Error loading {toTable}: {e}")
        raise

# Load all tables with proper error handling
tables_to_load = [
    ("DimCustomer", "DimCustomer"),
    ("DimDate", "DimDate"),
    ("DimProduct", "DimProduct"),
    ("FactInternetSales", "FactInternetSales")
]

for from_table, to_table in tables_to_load:
    loadDataToLakehouse(from_table, to_table)

print("Done")

## Step 5: Trigger Metadata Synchronisation

Force a synchronisation between the lakehouse storage layer and the SQL Analytics Endpoint. This ensures that the table schemas are up to date before creating the semantic model.

---

*Erzwingen Sie eine Synchronisation zwischen der Lakehouse-Speicherschicht und dem SQL Analytics Endpoint. Dadurch wird sichergestellt, dass die Tabellenschemata aktuell sind, bevor das Semantic Model erstellt wird.*

In [ ]:
##https://medium.com/@sqltidy/delays-in-the-automatically-generated-schema-in-the-sql-analytics-endpoint-of-the-lakehouse-b01c7633035d

def triggerMetadataRefresh():
    client = fabric.FabricRestClient()
    response = client.get(f"/v1/workspaces/{workspaceId}/lakehouses/{lakehouseId}")
    sqlendpoint = response.json()['properties']['sqlEndpointProperties']['id']

    # trigger sync
    uri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}"
    payload = {"commands":[{"$type":"MetadataRefreshExternalCommand"}]}
    response = client.post(uri,json= payload)
    batchId = response.json()['batchId']

    # Monitor Progress
    statusuri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}/batches/{batchId}"
    statusresponsedata = client.get(statusuri).json()
    progressState = statusresponsedata['progressState']
    print(f"Metadata refresh : {progressState}")
    while progressState != "success":
        statusuri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}/batches/{batchId}"
        statusresponsedata = client.get(statusuri).json()
        progressState = statusresponsedata['progressState']
        print(f"Metadata refresh : {progressState}")
        time.sleep(1)

    print('Metadata refresh complete')

triggerMetadataRefresh()

## Step 6: Create the Direct Lake Semantic Model

Generate a new Direct Lake semantic model from the lakehouse tables. The model is created using the DL/SQL connection type, which references the SQL Analytics Endpoint.

---

*Erstellen Sie ein neues Direct Lake Semantic Model aus den Lakehouse-Tabellen. Das Modell wird mit dem DL/SQL-Verbindungstyp erstellt, der auf den SQL Analytics Endpoint verweist.*

In [ ]:
from sempy import fabric

#1. Generate list of ALL table names from lakehouse to add to Semantic Model
lakehouseTables:list = labs.lakehouse.get_lakehouse_tables(lakehouse=LakehouseName)["Table Name"]

completedOK:bool=False
while not completedOK:
    try:
        #2 Create the semantic model
        if sempy.fabric.list_items().query(f"`Display Name`=='{LakehouseName}_model' & Type=='SemanticModel'  ").shape[0] ==0:
            labs.directlake.generate_direct_lake_semantic_model(dataset=f"{LakehouseName}_model",lakehouse_tables=lakehouseTables,workspace=workspaceName,lakehouse=lakehouseId,refresh=False,overwrite=True)
            completedOK=True
    except:
        print('Error creating model... trying again.')
        time.sleep(3)
        triggerMetadataRefresh()

print('Semantic model created OK')

## Step 7: Configure Table Relationships

Define star schema relationships linking the fact table (FactInternetSales) to its dimension tables (DimCustomer, DimDate, DimProduct). These relationships enable cross-table filtering in DAX queries.

---

*Definieren Sie Sternschema-Beziehungen, die die Faktentabelle (FactInternetSales) mit ihren Dimensionstabellen (DimCustomer, DimDate, DimProduct) verknuepfen. Diese Beziehungen ermoeglichen tabellenuebergreifendes Filtern in DAX-Abfragen.*

In [ ]:
completedOK:bool=False
while not completedOK:
    try:
        with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
            #1. Remove any existing relationships
            for r in tom.model.Relationships:
                tom.model.Relationships.Remove(r)

            #2. Creates correct relationships
            tom.add_relationship(from_table="FactInternetSales", from_column="OrderDateKey" , to_table="DimDate"    , to_column="DateKey"       , from_cardinality="Many" , to_cardinality="One")
            tom.add_relationship(from_table="FactInternetSales", from_column="CustomerKey"  , to_table="DimCustomer", to_column="CustomerKey"   , from_cardinality="Many" , to_cardinality="One")
            tom.add_relationship(from_table="FactInternetSales", from_column="ProductKey"   , to_table="DimProduct" , to_column="ProductKey"    , from_cardinality="Many" , to_cardinality="One")
            completedOK=True
    except:
        print('Error adding relationships... trying again.')
        time.sleep(3)

print('done')


## Step 8: Add Business Measures

Create a set of DAX measures (such as Total Sales, Order Count, and Average Order Value) to provide meaningful aggregations for reporting and validation later in this lab.

---

*Erstellen Sie eine Reihe von DAX-Measures (wie Gesamtumsatz, Bestellanzahl und durchschnittlicher Bestellwert), um aussagekraeftige Aggregationen fuer das Reporting und die spaetere Validierung in diesem Lab bereitzustellen.*

In [ ]:
completedOK:bool=False
while not completedOK:
    try:
        with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
            #1. Remove any existing measures
            for t in tom.model.Tables:
                for m in t.Measures:
                    tom.remove_object(m)
                    print(f"[{m.Name}] measure removed")

            tom.add_measure(table_name="FactInternetSales" ,measure_name="Sum of Sales",expression="SUM(FactInternetSales[SalesAmount])",format_string="\$#,0.###############;(\$#,0.###############);\$#,0.###############")
            tom.add_measure(table_name="FactInternetSales" ,measure_name="Count of Sales",expression="COUNTROWS(FactInternetSales)",format_string="#,0")
            completedOK=True
    except:
        print('Error adding measures... trying again.')
        time.sleep(3)

print('done')

## 9. Configure Date Table for Time Intelligence

Marks DimDate table as date table to enable time-based analysis functions and calendar features.

In [ ]:
completedOK:bool=False
while not completedOK:
    try:
        with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
            tom.mark_as_date_table(table_name="DimDate",column_name="Date")
            completedOK=True
    except:
        print('Error with date table... trying again.')
        time.sleep(3)

print('done')

## Step 10: Configure Column Sorting

Set sort-by-column properties on date table columns so that month names and other textual date fields appear in chronological order rather than alphabetical order in visualisations.

---

*Legen Sie Sortierungseigenschaften fuer Datumstabellenspalten fest, damit Monatsnamen und andere textuelle Datumsfelder in Visualisierungen in chronologischer statt alphabetischer Reihenfolge angezeigt werden.*

In [ ]:
import json
tom = labs.tom.TOMWrapper(dataset=SemanticModelName, workspace=workspaceName, readonly=False)
tom.set_sort_by_column(table_name="DimDate",column_name="MonthName"       ,sort_by_column="MonthNumberOfYear")
tom.set_sort_by_column(table_name="DimDate",column_name="DayOfWeek"       ,sort_by_column="DayNumberOfWeek")
tom.model.SaveChanges()

i:int=0
for t in tom.model.Tables:
    if t.Name=="DimDate":
        bim = json.dumps(tom.get_bim()["model"]["tables"][i],indent=4)
        print(bim)
    i=i+1

## Step 9: Configure Date Table for Time Intelligence

Mark the DimDate table as a date table. This enables DAX time intelligence functions such as year-over-year comparisons and running totals.

---

*Markieren Sie die DimDate-Tabelle als Datumstabelle. Dies aktiviert DAX-Zeitintelligenzfunktionen wie Jahresvergleiche und laufende Summen.*

In [ ]:
i:int=0
for t in tom.model.Tables:
    if t.Name in ["FactInternetSales"]:
        for c in t.Columns:
            c.IsHidden=True

        bim = json.dumps(tom.get_bim()["model"]["tables"][i],indent=4)
        print(bim)
    i=i+1
    
tom.model.SaveChanges()

## Step 11: Hide Fact Table Columns

Hide the raw columns on the fact table to guide report authors towards using the defined measures instead of dragging raw columns into visualisations.

---

*Blenden Sie die Rohspalten der Faktentabelle aus, um Berichtsautoren dazu zu leiten, die definierten Measures zu verwenden, anstatt Rohspalten in Visualisierungen zu ziehen.*

In [ ]:
reframeOK:bool=False
while not reframeOK:
    try:
        result:pandas.DataFrame = labs.refresh_semantic_model(dataset=SemanticModelName)
        reframeOK=True
    except:
        print('Error with reframe... trying again.')
        triggerMetadataRefresh()
        time.sleep(3)

print('Custom Semantic Model reframe OK')

## Step 12: Frame the Semantic Model

Trigger framing on the semantic model so that it is ready to serve queries. Framing establishes the initial connection between the model and the underlying Delta tables.

---

*Loesen Sie das Framing des Semantic Models aus, damit es bereit ist, Abfragen zu bedienen. Framing stellt die initiale Verbindung zwischen dem Modell und den zugrunde liegenden Delta-Tabellen her.*

In [ ]:
import warnings
import time
from Microsoft.AnalysisServices.Tabular import TraceEventArgs
from typing import Dict, List, Optional, Callable

def runDMV():
    df = sempy.fabric.evaluate_dax(
        dataset=SemanticModelName, 
        dax_string="""
        
        SELECT 
            MEASURE_GROUP_NAME AS [TABLE],
            ATTRIBUTE_NAME AS [COLUMN],
            DATATYPE ,
            DICTIONARY_SIZE 		    AS SIZE ,
            DICTIONARY_ISPAGEABLE 		AS PAGEABLE ,
            DICTIONARY_ISRESIDENT		AS RESIDENT ,
            DICTIONARY_TEMPERATURE		AS TEMPERATURE,
            DICTIONARY_LAST_ACCESSED	AS LASTACCESSED 
        FROM $SYSTEM.DISCOVER_STORAGE_TABLE_COLUMNS 
        ORDER BY 
            [DICTIONARY_TEMPERATURE] DESC
        
        """)
    display(df)

## Step 13: Set Up DMV Monitoring Function

Define a helper function that queries Dynamic Management Views (DMVs) to inspect column temperature and memory usage. This function will be used in the following steps to observe how Direct Lake loads data on demand.

---

*Definieren Sie eine Hilfsfunktion, die Dynamic Management Views (DMVs) abfragt, um Spaltentemperatur und Speichernutzung zu ueberpruefen. Diese Funktion wird in den folgenden Schritten verwendet, um zu beobachten, wie Direct Lake Daten bei Bedarf laedt.*

In [ ]:
df=sempy.fabric.evaluate_dax(
    dataset=SemanticModelName, 
    dax_string="""
    
    evaluate tabletraits()
    
    """)
display(df)

In [ ]:
## Step 14: Explore Direct Lake Table Traits and Guardrails

Use the TABLETRAITS() DAX function and guardrails queries to verify that the model is operating in Direct Lake mode and to review the applicable capacity limits.

---

*Verwenden Sie die DAX-Funktion TABLETRAITS() und Guardrails-Abfragen, um zu ueberpruefen, ob das Modell im Direct Lake-Modus arbeitet, und um die geltenden Kapazitaetsgrenzen zu pruefen.*

## Step 15: Establish a Performance Baseline with DMV Analysis

Capture the current state of Direct Lake columns and memory usage before running any queries. This provides a baseline to compare against after query execution.

---

*Erfassen Sie den aktuellen Zustand der Direct Lake-Spalten und der Speichernutzung, bevor Abfragen ausgefuehrt werden. Dies bietet eine Basislinie fuer den Vergleich nach der Abfrageausfuehrung.*

In [ ]:
runDMV()

## Step 16: Execute a DAX Query and Monitor Column Loading

Run a DAX query against the model and then use the DMV monitoring function to observe which columns were loaded into memory. This demonstrates Direct Lake's on-demand column loading behaviour.

---

*Fuehren Sie eine DAX-Abfrage gegen das Modell aus und verwenden Sie dann die DMV-Ueberwachungsfunktion, um zu beobachten, welche Spalten in den Speicher geladen wurden. Dies demonstriert das bedarfsgesteuerte Spaltenladeverhalten von Direct Lake.*

In [ ]:
labs.clear_cache(SemanticModelName)

df=sempy.fabric.evaluate_dax(
    dataset=SemanticModelName, 
    dax_string="""
    
    EVALUATE
        SUMMARIZECOLUMNS(
               
                DimDate[MonthName] ,
                "Count of Transactions" , COUNTROWS(FactInternetSales) ,
                "Sum of Sales" , [Sum of Sales] 
        )
        ORDER BY [MonthName]
    """)
display(df)

runDMV()

## Step 17: Stop the Spark Session

---

*Spark-Sitzung beenden.*

In [ ]:
mssparkutils.session.stop()

## Lab 1 Summary

### What You Accomplished

In this lab you built a complete Direct Lake semantic model from scratch:

- **Infrastructure:** Created a lakehouse and loaded four Adventure Works tables (approximately 80,000 rows in total)
- **Model creation:** Generated a Direct Lake semantic model with automatic table discovery
- **Data modelling:** Defined star schema relationships between the fact table and three dimension tables
- **Business logic:** Added DAX measures with appropriate formatting
- **User experience:** Configured date table marking, column sorting, and column visibility
- **Validation:** Used DMV queries to observe on-demand column loading and memory usage

### Architecture

```
Adventure Works Data --> Lakehouse (Delta Tables) --> Direct Lake Model --> Power BI Reports
```

### Key Takeaways

- Direct Lake queries data directly from Delta files, avoiding the need for scheduled imports
- Columns are loaded into memory only when a query requires them
- DMV queries provide visibility into which columns are loaded and how much memory is consumed
- Framing establishes the initial link between the semantic model and the underlying Delta tables

### Next Lab

Continue to **Lab 2** to work with billion-row datasets and OneLake shortcuts.

---

*Deutsche Version:*

### Was Sie erreicht haben

In diesem Lab haben Sie ein vollstaendiges Direct Lake Semantic Model von Grund auf erstellt:

- **Infrastruktur:** Ein Lakehouse erstellt und vier Adventure Works-Tabellen geladen (insgesamt ca. 80.000 Zeilen)
- **Modellerstellung:** Ein Direct Lake Semantic Model mit automatischer Tabellenerkennung generiert
- **Datenmodellierung:** Sternschema-Beziehungen zwischen der Faktentabelle und drei Dimensionstabellen definiert
- **Geschaeftslogik:** DAX-Measures mit entsprechender Formatierung hinzugefuegt
- **Benutzererfahrung:** Datumstabellenmarkierung, Spaltensortierung und Spaltensichtbarkeit konfiguriert
- **Validierung:** DMV-Abfragen verwendet, um das bedarfsgesteuerte Laden von Spalten und die Speichernutzung zu beobachten

### Wichtige Erkenntnisse

- Direct Lake fragt Daten direkt aus Delta-Dateien ab und vermeidet geplante Importe
- Spalten werden nur dann in den Speicher geladen, wenn eine Abfrage sie benoetigt
- DMV-Abfragen bieten Einblick in geladene Spalten und Speicherverbrauch
- Framing stellt die initiale Verbindung zwischen Semantic Model und Delta-Tabellen her

### Naechstes Lab

Weiter zu **Lab 2**, um mit Milliarden-Zeilen-Datensaetzen und OneLake-Shortcuts zu arbeiten.